# 使用S2cloudless制作数据集

In [ ]:
!pip install geemap
!pip install eerepr

In [ ]:
import ee 
import geemap
import numpy as np
import eerepr
import folium
import os
from functools import partial
ee.Authenticate()
ee.Initialize()

## 载入原始数据，并定义过滤器

In [ ]:
Map = geemap.Map(center=[34, 99], zoom=4, add_google_map=True)
Southest_Tibetan_01 = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/Southest')
Southest_Tibetan_02 = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/Southest2')
Map.addLayer(Southest_Tibetan_01, {}, 'Southest_Tibetan')
Map

In [ ]:
Southest_Tibetan_Geo = Southest_Tibetan_01.map(lambda feature:feature.set({'Geo':feature.geometry()}))
Southest_Tibetan_GeoList = ee.List(Southest_Tibetan_Geo.reduceColumns(ee.Reducer.toList(),['Geo']).get('list'))
Num_list = Southest_Tibetan_GeoList.size().getInfo()

In [ ]:
START_DATE = '2020-06-01'          # 
END_DATE = '2020-09-30'            # 
Tolerance = 0.9                    # 样本筛选宽容度
Tolerance_cloudPix = 500          # 合成样本空洞容忍量
CLOUD_FILTER = 60                  # 过滤s2 大于指定云量的数据
CLD_PRB_THRESH = 15                # s2cloudless 概率值阈值[0-100],原实验是50
NIR_DRK_THRESH = 0.15              # 非水暗像素判断阈值
CLD_PRJ_DIST = 1                   # 根据 CLD_PRJ_DIST 输入指定的距离从云中投射阴影
BUFFER = 50                        # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input

## 定义函数：连接图像与云图，添加云和云阴影(计算)波段，采用folium进行可视化

In [ ]:
def make_dir(path):
    isExists = os.path.exists(path)
    # 判断结果
    if not isExists:
        os.makedirs(path)
        print(path + ' 创建成功')
    return path
##--------------------------------------- 云检测以及云阴影检测------------------------------------------------
def get_s2_sr_cld_col(aoi, start_date, end_date):
    """筛选S2图像以及S2_cloud图像，并将两个collection连接"""
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)).map(partial(clip_AOI,AOI=aoi)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date).map(partial(clip_AOI,AOI=aoi)))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    # 固定用法，将两个collection通过属性值连接起来，s2cloudless整体作为一个属性写入
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    """Define a function to add the s2cloudless probability layer
    and derived cloud mask as bands to an S2 SR image input."""
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    """Define a function to add dark pixels, 
    cloud projection, and identified shadows as bands to an S2 SR image input.
    Note that the image input needs to be the result of the above add_cloud_bands function
    because it relies on knowing which pixels are considered cloudy ('clouds' band)."""
    # 从 SCL 波段识别水像素, 仅适用于L2A，采用L1C计算MNDWI
    not_water = img.select('SCL').neq(6)

    # 识别非水的暗 NIR 像素(潜在的云阴影像素)。.
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # 确定云投射云影的方向(假设是 UTM 投影)。
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # 根据 CLD_PRJ_DIST 输入指定的距离从云中投射阴影
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    """Define a function to assemble all of the cloud and cloud shadow components and produce the final mask."""
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)
#     return img.addBands(is_cld_shdw)

##--------------------------------------- 使用云图层更新-------------------------------------------------
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()
    # Subset reflectance bands and update their masks, return the result.
    return img.select(['B.*','clouds','dark_pixels','shadows','cloudmask']).updateMask(not_cld_shdw)

##---------------------------------------- folium可视化-------------------------------------------------
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


def display_cloud_layers(col,AOI):
    # Mosaic the image collection.使用掩码合成集合中的所有图像。普通的mosaic，镶嵌过程中是取最新的影像值
    img = col.mosaic().clip(AOI)

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.centroid求中心点
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
#     m.add_ee_layer(cloud_transform,
#                    {'min': 0, 'max': 1, 'palette': ['white', 'black']},
#                    'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

    
def display_cloudfree_layers(s2_sr_median,AOI):
    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(s2_sr_median.clip(AOI),
                    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                    'S2 cloud-free mosaic', True, 1, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

def clip_AOI(col,AOI):
    return col.clip(AOI)

# 提取dimensions等波段的参数
# Image = ee.Image(s2_sr_cld_col.toList(5).get(1))
# bandList = ee.Dictionary(ee.List(ee.Dictionary(ee.Algorithms.Describe(Image)).get('bands')).get(0)).get('dimensions')
def write_dimensions(col,AOI):
    dimensions = ee.Dictionary(ee.List(ee.Dictionary(ee.Algorithms.Describe(col)).get('bands')).get(0)).get('dimensions')
    allNoneZeros = col.select('B1').unmask(0).reduceRegion( **{
                        'reducer': ee.Reducer.allNonZero(),
                        'scale': 10,
                        'maxPixels': 1e12,
                        'bestEffort': True,
                    }).get('B1')
    
    all_NotNone = col.select('B1').unmask(0).reduceRegion(
                        **{
                            'geometry': AOI,
                            'reducer': ee.Reducer.count(),
                            'scale': 10,
                            'maxPixels': 1e12,
                            'bestEffort': True,
                        }).get('B1')
    
    return col.set({'dimensions':ee.List(dimensions).reduce(ee.Reducer.sum()),
                   'all_NotNone':all_NotNone})
    
def cloud_sum(col,AOI):

    cloud_count = col.select('clouds').reduceRegion(
                    **{
                        'geometry': AOI,
                        'reducer': ee.Reducer.sum(),
                        'scale': 10,
                        'maxPixels': 1e12,
                        'bestEffort': True,
                    }
                )
    return col.set({'cloud_count': cloud_count})

In [ ]:
# Diplay_test = s2_sr_cld_col_eval_disp.first()
# Map.addLayer(Diplay_test.select('clouds'),{ 'min': 0, 'max': 1},'clouds')
# Map.addLayer(Diplay_test.select("dark_pixels"),{ 'mi.n': 0, 'max': 1},"dark_pixels")
# Map.addLayer(Diplay_test.select("shadows"),{ 'min': 0, 'max': 1},"shadows")
# Map.addLayer(Diplay_test.select("cloudmask"),{'min': 0, 'max': 1},"cloudmask")
# Map.addLayer(Diplay_test,{ 'bands':["B4","B3","B2"],'min': 0, 'max': 10000},'S2_image')
# Map.addLayer(s2_sr_median,{ 'bands':["B4","B3","B2"],'min': 0, 'max': 2500,'gamma': 1.1},'S2 cloud-free mosaic')
# Map.addLayer(s2_sr_median,{ 'bands':["cloudmask"],'min': 0, 'max': 1,'gamma': 1.1},'S2 cloud mosaic')

## 生成无云的遥感影像，并采用folium进行可视化

## 判断采用的无云遥感影像，并导出数据

In [ ]:
out_dir = '/content/'
def Geemap_export(out_dir,collection=False,image=False,region=None,scale=10):
    if collection:
        # 这里导出时候使用region设置AOI，否则可能因为坐标系问题(未确定)，出现黑边问题
        geemap.ee_export_image_collection(collection,
                                  out_dir=out_dir,
                                  format = "ZIPPED_GEO_TIFF",region=region,scale=scale)
        print('collection save right')
    elif image:
        try:
            filename=os.path.join(out_dir,'Clear'+image.id().getInfo()+'.tif')
        except:
            filename=os.path.join(out_dir,'Cloud.tif')
            
        geemap.ee_export_image(image,
                       filename=filename,
                       scale=scale, region=region, file_per_band=True,timeout=1500)
        print('image save right')
    else:
        print('Erro:collection && image must have one False')

In [ ]:
import os,tarfile
from google.colab import files,drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive')
# out_dir = 'D:/GEE_S2Cloud_Dataset'


def make_targz_one_by_one(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  all_files = os.walk(source_dir)
  for root,dir_name,files_list in all_files:
    for file in files_list:
      if file.endswith('.tif'):
        pathfile = os.path.join(root, file)
        tar.add(pathfile)
  tar.close()
  files.download(output_filename)


def make_targz_all_path(output_filename,paths):
  tar = tarfile.open(output_filename,"w")
  for each in paths:
    tar.add(each)
  tar.close()

In [ ]:
Paths = []; Num = 0;
for i in range(518,Num_list):
    path_real = os.path.join(out_dir,START_DATE+'_'+END_DATE,'Real',f'{i:05d}')
    path_synth = os.path.join(out_dir,START_DATE+'_'+END_DATE,'Synth',f'{i:05d}')
    if os.path.exists(path_synth) or os.path.exists(path_real):
        print('第{}个切块已存在，跳过执行'.format(f'{i:05d}'))
        continue

    AOI = ee.Feature.geometry(Southest_Tibetan_GeoList.get(i))
    AOI_buffer = AOI.buffer(distance=300)  #用于图像clip，到处时候采用AOI不会出现黑边现象
    
    # 取数据
    s2_sr_cld_col = get_s2_sr_cld_col(AOI_buffer, START_DATE, END_DATE).map(partial(write_dimensions,AOI=AOI))
    # 根据Dimension大小删除不完整的数据
    try:
      Max_all_NotNone = s2_sr_cld_col.aggregate_array('all_NotNone').reduce(ee.Reducer.max())
      s2_sr_cld_col = s2_sr_cld_col.filter(ee.Filter.gte('all_NotNone', ee.Array(Max_all_NotNone).multiply(Tolerance).getInfo()))
    except:
      continue
#     Max_dimension = s2_sr_cld_col.aggregate_array('dimensions').reduce(ee.Reducer.max())
#     s2_sr_cld_col = s2_sr_cld_col.filter(ee.Filter.gte('dimensions', Max_dimension))
    
    # 将shadow和cloud图层加入波段
    s2_sr_cld_col_disp = s2_sr_cld_col.map(add_cld_shdw_mask).select(['B.*','clouds','dark_pixels','shadows','cloudmask'])
    s2_sr_cld_col_list = s2_sr_cld_col_disp.toList(s2_sr_cld_col_disp.size())
    # 计算AOI区段的云量
    cloud_count_dict = s2_sr_cld_col_disp.map(partial(cloud_sum,AOI=AOI)).aggregate_array('cloud_count').getInfo()
    cloud_count_list = np.array([1 if item.get('clouds')>Tolerance_cloudPix else 0 for item in cloud_count_dict])
    
    # 存在有无云影像的时候
    if np.min(cloud_count_list) == 0:
        #    save_dir置前，先检查本地文件夹是否存在

        '''执行方案一，直接获取对照组'''
        Real_dir = make_dir(path_real)
        Negtive_Indice = list(range(len(cloud_count_list)))
        Positive_Indice = Negtive_Indice.pop(cloud_count_list.argmin())

        # 选择正例
        Positive_Exm = ee.Image(s2_sr_cld_col_list.get(Positive_Indice))
        # 选择负例
        Negtive_Exm = ee.ImageCollection(s2_sr_cld_col_list.remove(Positive_Exm))
        
        # save
        Geemap_export(Real_dir,collection=False,image=Positive_Exm,region=AOI,scale=10)
        
        Geemap_export(Real_dir,collection=Negtive_Exm,image=False,region=AOI,scale=10)

        Paths += [os.path.join(Real_dir,f) for f in os.listdir(Real_dir) if f.endswith('.tif') ]
        
        print('第{}切块执行方案一，保存至{},SaveNum={}'.format(f'{i:05d}',Real_dir,Num))
    # 不存在无云影像的时候
    else:
        """执行方案二，构造对照组"""

        Synth_dir = make_dir(path_synth)
        s2_sr_median = s2_sr_cld_col_disp.map(apply_cld_shdw_mask).median().select('B.*').int16()
     
        # 合成影像中的云量
        cloudNUM = s2_sr_median.select('B1').unmask(0).lt(0.1).reduceRegion(
                        **{
                            'geometry': AOI,
                            'reducer': ee.Reducer.sum(),
                            'scale': 10,
                            'maxPixels': 1e12,
                            'bestEffort': True,
                        }
                    ).getInfo()['B1']

        if cloudNUM<=Tolerance_cloudPix:

            Geemap_export(Synth_dir,collection=False,image=s2_sr_median,region=AOI,scale=10)

            Geemap_export(Synth_dir,collection=s2_sr_cld_col_disp,image=False,region=AOI,scale=10)

            Paths += [os.path.join(Synth_dir,f) for f in os.listdir(Synth_dir) if f.endswith('.tif') ]
            print('第{}切块采用方案二,保存至{},SaveNum={}'.format(f'{i:05d}',Synth_dir,Num))
            
        else:
            print('第{}切块采用方案二,结果cloudNUM={}不满足云去除要求,SaveNum={}'.format(f'{i:05d}',cloudNUM,Num))
            
    Num += 1
    if Num % 25 == 0:
      print('当前Paths长度为{}保存数据并清空Paths'.format(len(Paths)))
      make_targz_all_path(f'{i:05d}.tar',Paths)
      files.download(f'{i:05d}.tar')
      Paths = []

In [ ]:
make_targz_all_path(f'{i:05d}.tar',Paths)
files.download(f'{i:05d}.tar')